### Chapter 7. Extending the OLG model
Reference: Fehr, H. and Kindermann, F. 2018. The overlapping generations model. Introduction to Computational Economics Using Fortran. Oxford University Press. pp253-286.

In [1]:
##### Program 7.3 Aggregation of individual variables with endogenous growth

import numpy as np

# Parameters
"""
    T: transition path periods
    J: age
    JR: the year household retires
    γ: risk aversion
    egam: denominator of utility function
    β: discount factor
    ν: leisure preference parameter
    ρ: intratemporal elasticity of substitution between consumption and leisure
    eρ: denominator of leisure preference
    α: capital share
    δ: depreciation
    tol: tolerance
    damp: speed of convergence
    itermax: maximum iteration number
    ξ: parameter of education function ξe^υ
    υ: parameter of education function ξe^υ
    ϵ: the strength o the externality in Y=(K^α)L^(1-α)H^ϵ
    μ: shadow price
    ----------------------------------------------------------------------------
    w: wage rate
    r: rate of capital return
    wn: after tax wage (net wage)
    winc: wage income
    Rn: after tax capital return
    p: consumption price
    K: aggregate capital
    L: aggregate labour
    A: aggregate assets
    C: aggregate consumption
    I: aggregate investment
    B: aggregate debt
    G: government spending
    k: capital per capita
    a: savings
    e: education investment
    c: consumption
    U: utility
    l: leisure
    h: household productivity (human capital)
    Y: aggregate output
    Ba: debt or assets of the agency per capita of workers
    Bf: future period debt B_{t+1}
    Tb: total debt (1+np)B_{t+1}+(1+r)B_t
    TXr: total tax revenue
    Tpen: total pension
    H: the average stock o human capitak per worker
    
"""

def initialize():
    global T, J, JR, γ, egam, β, α, δ, ξ, υ, ϵ, μ, tol, damp, itermax,\
    w, r, wn, winc, Rn, p, K, ma, n_e,\
    L, A, C, I, B, G, k, Y, Bf, Tb, TXr, Tpen, H, n_p, gy, by, κ,\
    smopec, τk, τc, τw, τr, τp, τs, tax, pen, a, e, c, m

    # Parameters
    T = 24; J = 3; JR = 3; γ = .5; egam = 1.0 - 1.0/γ; β = .9
    α = .3; δ = .0; ξ = 2.0; υ = 0.5; ϵ = 0.0
    μ = 0.8; tol = 1e-6; damp = 0.3; itermax = 200

    # Variables
    w  = np.zeros(T+1); r = np.zeros(T+1); wn = np.zeros(T+1)
    winc = np.zeros((J,T+1)); ma = np.zeros((J,T+1))
    Rn = np.zeros(T+1); p = np.zeros(T+1); K  = np.zeros(T+1)
    L  = np.zeros(T+1); A = np.zeros(T+1); C  = np.zeros(T+1)
    I  = np.zeros(T+1); B = np.zeros(T+1); G  = np.zeros(T+1)
    k  = np.zeros(T+1); Y = np.zeros(T+1); Bf = np.zeros(T+1)
    Tb = np.zeros(T+1); TXr = np.zeros(T+1); Tpen = np.zeros(T+1)
    H = np.ones(T+1)

    # Set model parameters
    n_p = np.zeros(T+1)         #population growth rate
    n_e = np.zeros(T+1)         #endogenous growth rate in human capital (productivity)
    gy  = 0.0                   #government spending proportion
    by  = np.zeros(T+1)         #debt proportion
    κ   = np.zeros(T+1)         #pension proportion
    smopec = False              #include small open economy or not

    # Initialize tax rates shadow wages and pensions
    τk  = np.zeros(T+1)         #investment tax rate
    τc = np.zeros(T+1)          #consumption tax rate
    τw = np.zeros(T+1)          #labour income tax rate
    τr = np.zeros(T+1)          #capital income tax rate
    τp = np.zeros(T+1)          #payroll tax rate
    τs = np.zeros(T+1)          #subsidy rate of the government for education costs
    tax = np.ones(T+1)          #tax balancing index
    pen = np.zeros((J,T+1))     #pension

    # Initialize assets
    a = np.zeros((J,T+1))       #assets
    e = np.zeros(T+1)           #education investment
    c = np.zeros((J,T+1))       #consumption

    # Size of cohorts in specific year
    m = np.zeros((J,T+1))       #population parameter
    for t in range(T+1):
        m[0,t] = 1.0            #in period 0, m=1
        tm = year(t, 1, 0)      #time index t-1
        for j in range(1,J):
            m[j,t] = m[j-1,tm]/(1.0 + n_p[t]) #m_{j,t}=m_{j-1,t-1}/(1+np)
    return None

In [2]:
## calculates year at which age j agent is jp
def year(t, j, jp):
    """This function defines the year index"""
    year = t + jp - j  #year i age j period ij (j-1, j+1, j+2)
    if t == 0 or year<=0:
        year = 0       #initial period
    if t == T or year>=T:
        year = T       #last period
    return year

In [3]:
## Function: factor price:
def factor_prices(t):
    """This function defines multidimensional factor price calculation for period i."""
    global k, r, w, wn, Rn, p
    k[t]   = K[t]/L[t]                  #per capita capital stock
    if smopec and t > 0:
        r[t] = r[0]
    else:
        r[t] = (1.0 - τk[t])*(α*k[t]**(α-1.0)*H[t]**ϵ-δ)  #(1-τk)*r
  
    w[t]   = (1.0 - α)*k[t]**α*H[t]**ϵ  #wage rate

    wn[t]  = w[t]*(1.0 - τw[t] - τp[t]) #after tax wage rate
    winc[0,t] = (1.0-(1.0-τs[t])*e[t])*wn[t]   #wage income
    
    for j in range(1, JR-1):
        tp = year(t, j , 1)             #t+1-j
        winc[j,t] = wn[t]*(1.0+n_e[tp])/μ      #wn(1+n_e)/μ
    
    Rn[t]  = 1.0 + r[t]*(1.0 - τr[t])   #net interest
    p[t]   = 1.0 + τc[t]                #consumer price
    return None

In [4]:
## present value of resources for household aged j in year t

def get_W(j, t):
    """This function calculates the total income at each age j at period t."""
    # get current value of resources
    Assets = winc[j,t] + pen[j,t]       #initial total income include the income and pension
    
    if t == 1 and j > 0:                #if period 1 and age 1,2,...
        Assets = Assets + Rn[t] * a[j,t]  #initial assets + capital return
    
    # Iterate over remainder of life span
    PRn = 1.0
    for jp in range(j+1,J):  #loop for remainder of life span
        tp = year(t,j,jp)    #time index for remainder of life span
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        Assets = Assets + (winc[jp,tp] + pen[jp,tp])/PRn   #W equation above (7.18)
    return Assets


## marginal consumption for household aged j in year t

def get_Psi(j,t):
    """This function calculates the consumption proportion at each age j at period t."""
    Psi = 1.0         #set initial value of Psi
    PRn = 1.0         #PRn in the period 0 is 1
    
    for jp in range(j+1,J):  #loop for each future ages
        tp = year(t, j ,jp)  #year interval t+jp-j
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        Psi = Psi + β**((jp-j)*γ)*(p[tp]/(PRn*p[t]))**(1.0-γ)  #part inside big bracket of (7.16)
    Psi = 1.0/(p[t]*Psi)     #v/p/(big bracket) equation (7.16) with v=1 since leisure preference ν=0
    
    return Psi

In [5]:
## Calculating the optimal consumption path

def get_path(j,t):
    """This function calculates the optimal consumption path at each age j at period t."""
    
    global c, a
    
    PRn = 1.0         #PRn in the period 0 is 1

    for jp in range(j+1,J):  #loop for future ages (j+1,...,J)
        # get future and previous year as well as interest factor
        tp = year(t,j,jp)    #t+jp-j
        tm = year(t,j,jp-1)  #t+jp-j-1
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        
        # get consumption and leisure
        c[jp,tp] = (β**(jp-j)*PRn*p[t]/p[tp])**γ*c[j,t]      #c_{2,t+1} and c_{3,t+2} equation (7.13&14)
        # calculate assets 
        a[jp,tp] = winc[jp-1,tm] + pen[jp-1,tm] + Rn[tm]*a[jp-1,tm] - p[tm]*c[jp-1,tm]
        #savings as all income (labour income + pension + capital income) - consumption
        
    return None

In [6]:
## Calculating individual decisions in a certain year

def decisions(t):
    """This function defines the decisions of consumption and savings in each period t."""
    global c, e
    
    tp = year(t, 0, 1)                 #t+1
    e[t] = (ξ*υ*wn[tp]/(wn[t]*(1.0-τs[t])*Rn[tp]))**(1.0/(1.0-υ))  #equation (7.19)
    n_e[tp] = μ*(1.0+ξ*e[t]**υ) - 1.0  #endogenous growth rate equation (7.23)
    
    c[0,t] = get_Psi(0,t)*get_W(0,t) #c=ΨW
    get_path(0,t)                    #calculate consumption path at age 0 period t
    
    # derive behaviour for all other cohorts in year 1 of transition
    if t == 1:                       #at t=1
        for j in range(1,J):         #for each age
            c[j,t] = get_Psi(j,t)*get_W(j,t)    #c=ψW
            get_path(j,t)                       #calculate consumption path at age j period t
    return None

In [7]:
## calculating quantities in a certain year

def quantities(t):
    """This function defines the aggregated quantities of C, A, H, Y, I, G, B, K, Bf, Tb, I, ma in period t."""
    global G, C, A, L, H, Y, B, K, Bf, Tb, I, ma
    
    tm = year(t,1,0)   #time index t-1
    tp = year(t,0,1)   #time index t+1
    
    if t == 0:         #initial goverment spending
        G[t] = gy*Y[t] #goverment spending
    else:              #other periods
        G[t] = G[0]    #same as period 0
        
    ma[0,t] = 1.0      #initial population parameter
    for j in range(1,J):  #update age
        ma[j,t] = ma[j-1,tm]/((1.0+n_e[t])*(1.0+n_p[t]))   #population parameter ma=m_{t-1}/(1+ne)(1+np)
        
    # aggregate individual decisions
    C[t] = .0          #initial value of aggregate consumption
    A[t] = .0          #initial value of aggregate assets
    L[t] = .0          #initial value of aggregate labour input
    H[t] = .0          #initial value of aggregate human capital stock (productivity)
    
    for j in range(J): #for all ages
        C[t] = C[t] + c[j,t]*ma[j,t] #cumulated aggregate consumption with population parameters ma=m_{t-1}/(1+ne)(1+np)
        A[t] = A[t] + a[j,t]*ma[j,t] #cumulated aggregate assets with population parameters ma=m_{t-1}/(1+ne)(1+np)
        if j < JR-1:                 #if not retired
            L[t] = L[t] + μ**(-j)*m[j,t]  #cumulated aggregate labour with population parameters mt=m_{t-1}/(1+np)
            H[t] = H[t] + m[j,t]     #cumulated aggregate human capital stock with population parameters mt=m_{t-1}/(1+np)
            
    L[t] = L[t] - e[t]               #aggregate labour input
    H[t] = L[t]/(H[t] - e[t])        #aggregate human capital
    
    Y[t] = K[t]**α*L[t]**(1.0-α)*H[t]**ϵ  #aggregate output
    B[t] = by[tm]*Y[t]                    #aggregate debt
    
    # derive capital in small open or closed economy
    if smopec and t>0:
        K[t]  = L[t]*((r[t]*(1.0-eps[t]*τk[t])/(1.0-τk[t]) + δ)/α)**(1.0/(α-1.0)) #converted from r=α(K/L)^(α-1)H^ϵ-δ 
        Bf[t] = A[t] - K[t] - B[t]                                     #debt constraint
        Tb[t] = (1.0+n_p[tp])*(1.0+n_e[tp])*Bf[tp] - (1.0+r[t])*Bf[t]  #total debt
    else:
        K[t] = damp*(A[t]-B[t]) + (1.0-damp)*K[t]               #converging capital stock
    
    I[t] = (1.0+n_p[tp])*(1.0+n_e[tp])*K[tp] - (1.0 - δ)*K[t]   #aggregate investment
    
    return None

In [8]:
# calculating government parameters

def government(t):
    """This function defines the government parameters and tax revenues at period t."""
    global τc, τk, τw, τr, pen, Tpen, τp, TXr
    tp = year(t, 0, 1)             #t+1
    
    taxrev = np.zeros(5)           #set 5 types of taxes
    
    taxrev[0] = τc[t]*C[t]         #total consumption tax
    taxrev[1] = τw[t]*w[t]*L[t]    #total labour income tax
    taxrev[2] = τr[t]*r[t]*A[t]    #total capital income tax (on interest return)
    taxrev[3] = τk[t]*(Y[t] - w[t]*L[t]-(δ)*K[t])  #total savings or investment tax
    taxrev[4] = τs[t]*e[t]*wn[t]   #total education subsidy
    
    n_g = (1.0+n_p[tp])*(1.0+n_e[tp]) -1.0   #total growth rate
    
    # get budget balancing tax rate
    if tax[t] == 1:      #(government surplus-τcC)/C
        τc[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[1] + taxrev[2] + \
                    taxrev[3]+(1.0 + n_g)*B[tp]))/C[t]   
    elif tax[t] == 2:    #[government surplus-τw(wL)-τr(rA)]/(wL+rA) assuming same labour and capital income tax
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[3] + \
                    (1.0+n_g)*B[tp]))/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]    #same labour and capital income tax
    elif tax[t] == 3:    #[government surplus-τw(WL)]/(wL)
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[2] + \
                    taxrev[3] + (1.0 + n_g)*B[tp]))/(w[t]*L[t])
    else:                #[government surplus-τr(rA)]/(rA)
        τr[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[1] + \
                    taxrev[3]+ (1.0 + n_g)*B[tp]))/(r[t]*A[t])
    
    TXr[t] = np.sum(taxrev[0:4]) - taxrev[4]
    
    # get budget balancing social security contribution
    pen[JR-1:J,t] = κ[t]*w[t] #pension at each period
    Tpen[t] = .0              #initial total pension
    for j in range(JR-1,J):   #for retired periods
        Tpen[t] = Tpen[t] + pen[j,t]*ma[j,t] #total pension with population parameters mt=m_{t-1}/(1+ne)(1+np)
        
    τp[t] = Tpen[t]/(w[t]*L[t]) #payroll tax rate
    return None

In [9]:
# solves inital steady state using Gauss-Seidel

def get_SteadyState():
    """This function calculates inital steady state using Gauss-Seidel."""
    K[0] = 1.0  #initial capital
    L[0] = 1.0  #initial labour
    
    for i in range(itermax+1):
        factor_prices(0)
        decisions(0)
        quantities(0)
        government(0)
        if abs(Y[0]-C[0]-I[0]-G[0])/Y[0] < tol:
            break
    if i < itermax:
        print('Iteration: ', i, ' Diff: ',abs(Y[0]-C[0]-I[0]-G[0])/Y[0])
    else:
        print('!!! No equilibrium found !!!')
    
    return None

In [10]:
# sovles for transition path using Gauss-Seidel:

def get_Transition():
    """This function calculates each period economy and checks for market clearing conditions."""
    global a, K, L, HEV, H, h, Δ
    # Initialize values from initial equilibrium
    a[:,1] = a[:,0]   #saving
    n_e[1] = n_e[0]   #endogenous growth rate
    K[:] = K[0]       #aggregate capital
    L[:] = L[0]       #aggregate labour
    H[:] = H[0]       #aggregate human capital
    
    for ite in range(itermax+2):
        # get prices, decisions and quantites
        for t in range(T+1):
            factor_prices(t)
        
        for t in range(T+1):
            decisions(t)
        
        for t in range(T+1):
            quantities(t)
        
        for t in range(T+1):
            government(t)
            
        # check for the number of markets in equilibrium
        nmarket = 0
        for t in range(1,T+1):
            if  abs(Y[t]-C[t]-I[t]-G[t]-Tb[t])/Y[t] < tol:
                nmarket = nmarket + 1
        if nmarket == T:
            break
        
    if ite > itermax:
        print('!!! No equilibrium found !!!')
    else:
        print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    return None

In [11]:
# Table 7.8
initialize()
gy = .0
μm = [1.2, 1.0, 0.8]
Cm = np.zeros(3);em = np.zeros(3);nem = np.zeros(3);Lm = np.zeros(3);Km = np.zeros(3)
wm = np.zeros(3);rm = np.zeros(3)

for i in range(3):
    μ = μm[i]
    get_SteadyState()
    Cm[i] = C[0];em[i] = e[0];nem[i] = n_e[0];Lm[i] = L[0];Km[i] = K[0];wm[i] = w[0]
    rm[i] = r[0]

print('--------------------------------------')
print('Table 7.8 Long-run analysis of\nhuman-capital acquisition')
print('--------------------------------------')
print(' μ    C    e   ne    L    K    w    r')
print('--------------------------------------')
for i in range(3):
    print('%.1f'%μm[i],'%.2f'%Cm[i],'%.2f'%em[i],'%.2f'%nem[i],'%.2f'%Lm[i],'%.2f'%Km[i],'%.2f'%wm[i],'%.2f'%rm[i])

Iteration:  32  Diff:  8.22941653956756e-07
Iteration:  30  Diff:  8.458596542501232e-07
Iteration:  28  Diff:  9.042060961971886e-07
--------------------------------------
Table 7.8 Long-run analysis of
human-capital acquisition
--------------------------------------
 μ    C    e   ne    L    K    w    r
--------------------------------------
1.2 0.62 0.07 0.86 1.76 0.08 0.27 2.66
1.0 0.73 0.09 0.60 1.91 0.10 0.29 2.31
0.8 0.91 0.11 0.34 2.14 0.15 0.31 1.96


In [12]:
# Table 7.9
initialize()
μ = .8
gy = .22
tax[1:] = 3
get_SteadyState()
get_Transition()
print('-----------------------------------------------')
print('Table 7.9 From consumption to labour-income\ntaxation with endogenous growth')
print('-----------------------------------------------')
print(' t   τw   τc    C    e   ne    L    K    w    r ')
print('-----------------------------------------------')
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
for t in [T]:
    print(' T','%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print('-----------------------------------------------')

Iteration:  29  Diff:  8.107432976544846e-07
Iteration:  34  Markets:  24  Diff:  8.656958906270022e-07
-----------------------------------------------
Table 7.9 From consumption to labour-income
taxation with endogenous growth
-----------------------------------------------
 t   τw   τc    C    e   ne    L    K    w    r 
-----------------------------------------------
 0 0.00 0.30 0.70 0.11 0.34 2.14 0.15 0.31 1.96
 1 0.31 0.00 0.75 0.08 0.34 2.17 0.15 0.31 1.98
 2 0.33 0.00 0.68 0.08 0.25 2.17 0.12 0.29 2.25
 3 0.34 0.00 0.66 0.08 0.26 2.17 0.11 0.29 2.36
 4 0.34 0.00 0.65 0.08 0.25 2.17 0.11 0.28 2.44
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 T 0.35 0.00 0.63 0.08 0.25 2.17 0.10 0.28 2.54
-----------------------------------------------


In [13]:
# Table 7.10
# Table 7.9
initialize()
μ = .8
gy = .22
τs[1:] = .25
get_SteadyState()
get_Transition()
print('-----------------------------------------------')
print('Table 7.10 Education subsidies with endogenous growth')
print('-----------------------------------------------')
print(' t   τs   τc    C    e   ne    L    K    w    r ')
print('-----------------------------------------------')
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
for t in [T]:
    print(' T','%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print('-----------------------------------------------')

Iteration:  29  Diff:  8.107432976544846e-07
Iteration:  19  Markets:  24  Diff:  8.203982977455009e-07
-----------------------------------------------
Table 7.10 Education subsidies with endogenous growth
-----------------------------------------------
 t   τs   τc    C    e   ne    L    K    w    r 
-----------------------------------------------
 0 0.00 0.30 0.70 0.11 0.34 2.14 0.15 0.31 1.96
 1 0.25 0.33 0.68 0.17 0.34 2.08 0.15 0.32 1.92
 2 0.25 0.34 0.65 0.17 0.46 2.08 0.13 0.30 2.09
 3 0.25 0.36 0.63 0.17 0.46 2.08 0.12 0.30 2.19
 4 0.25 0.36 0.62 0.17 0.46 2.08 0.12 0.30 2.23
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 T 0.25 0.36 0.61 0.17 0.45 2.08 0.12 0.29 2.27
-----------------------------------------------
